In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
import pickle
import pandas as pd
import numpy as np

post_df = pickle.load( open( "/content/gdrive/My Drive/Colab Notebooks/labels_instagram_posts_13Nov.p", "rb" ) )

In [0]:
post_df.shape

(472, 6)

In [0]:
post_df.head()

,Unnamed: 0,post_url,num_likes,num_comments,caption,labels
0,0,https://scontent-sea1-1.cdninstagram.com/vp/8e...,455787,1234,Photo by Jimmy Chin @jimmychin | Half Dome aft...,"[Mountainous landforms, Mountain, White, Black..."
1,1,https://scontent-sea1-1.cdninstagram.com/vp/51...,246999,941,Photos by Ian Teh @iantehphotography | I’m on ...,"[Motor vehicle, Vehicle, Mode of transport, Au..."
2,2,https://scontent-sea1-1.cdninstagram.com/vp/21...,610340,3481,Photo by Thomas Peschak @thomaspeschak | The m...,"[Green sea turtle, Turtle, Sea turtle, Reptile..."
3,3,https://scontent-sea1-1.cdninstagram.com/vp/78...,160973,492,Photo by Katie Orlinsky @katieorlinsky | Olga ...,"[Temple, Cuisine]"
4,4,https://scontent-sea1-1.cdninstagram.com/vp/03...,185601,730,"Photo by Tasneem Alsultan @TasneemAlsultan | ""...","[Tree, Palm tree, Arecales, Woody plant, Plant..."


In [0]:
post_df['num_likes'] = post_df.num_likes.map(lambda x: float(x)/post_df.num_likes.max())
post_df['num_comments'] = post_df.num_comments.map(lambda x: float(x)/post_df.num_comments.max())

In [0]:
post_df['engagement'] = 0.4*post_df['num_likes']+0.6*post_df['num_comments']

In [0]:
post_df['e_lev'] = post_df.engagement.map(lambda x: 1 if x > post_df.engagement.median() else 0)

In [0]:
post_df['caption'] = post_df.caption.map(lambda x: x.lower())

In [0]:
post_df['labels'] = post_df.labels.map(lambda x: [i.lower() for i in x])

# Labels

In [0]:
reg_df = post_df[['labels','e_lev']]

In [0]:
reg_df.head()

,labels,e_lev
0,"[mountainous landforms, mountain, white, black...",1
1,"[motor vehicle, vehicle, mode of transport, au...",0
2,"[green sea turtle, turtle, sea turtle, reptile...",1
3,"[temple, cuisine]",0
4,"[tree, palm tree, arecales, woody plant, plant...",0


In [0]:
labels = list(set([i for l in post_df.labels for i in l]))

In [0]:
for l in labels:
  reg_df[l] = reg_df.labels.map(lambda x: 1 if l in x else 0) 

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression

X = reg_df.drop(columns=['e_lev','labels'])
Y = reg_df['e_lev']

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.20, random_state=11)
log_reg = LogisticRegression(random_state = 20,class_weight = 'balanced')
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
accuracy_score(y_test, y_pred)

0.7157894736842105

# Labels tf-idf

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')


import scipy
from scipy import sparse

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
reg_tfidf = post_df[['labels','e_lev']]

reg_tfidf['labels_join'] = reg_tfidf.labels.map(lambda x: ' '.join(x))

In [0]:
X = reg_tfidf.drop(columns=['e_lev'])
Y = reg_tfidf['e_lev']

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.20, random_state=11)
log_reg = LogisticRegression(random_state = 20,class_weight = 'balanced')


In [0]:
tf_idf = TfidfVectorizer(max_features=300, analyzer='word', stop_words='english', dtype=np.float32, ngram_range=(1,3))

# lemmatized_tr_l = X_train.labels_join.map(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))
# lemmatized_ts_l = X_test.labels_join.map(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))

# tr = tf_idf_l.fit_transform(lemmatized_tr_l)#X_train['labels_join'])
# ts = tf_idf_l.transform(lemmatized_ts_l)

tr = tf_idf.fit_transform(X_train['labels_join'])
ts = tf_idf.transform(X_test['labels_join'])

In [0]:
log_reg.fit(tr, y_train)
y_pred = log_reg.predict(ts)
accuracy_score(y_test, y_pred)

0.7263157894736842

# Captions tf-idf

In [0]:
reg_cap_tfidf = post_df[['caption','e_lev']]

X = reg_cap_tfidf.drop(columns=['e_lev'])
Y = reg_cap_tfidf['e_lev']

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.20, random_state=11)
log_reg = LogisticRegression(random_state = 20,class_weight = 'balanced')

In [0]:
tf_idf = TfidfVectorizer(max_features=3000, analyzer='word', stop_words= 'english', ngram_range=(1,3), dtype=np.float32)

tr = tf_idf.fit_transform(X_train['caption'])
ts = tf_idf.transform(X_test['caption'])

In [0]:
log_reg.fit(tr, y_train)
y_pred = log_reg.predict(ts)
accuracy_score(y_test, y_pred)

0.7684210526315789

# Captions lemmatize tf-idf

In [0]:
# https://www.machinelearningplus.com/nlp/lemmatization-examples-python/
import spacy

# Initialize spacy 'en' model, keeping only tagger component needed for lemmatization
nlp = spacy.load('en', disable=['parser', 'ner'])


In [0]:
reg_cap_tfidf = post_df[['caption','e_lev']]

X = reg_cap_tfidf.drop(columns=['e_lev'])
Y = reg_cap_tfidf['e_lev']

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.20, random_state=11)
log_reg = LogisticRegression(random_state = 20,class_weight = 'balanced')

In [0]:
lemmatizer = WordNetLemmatizer()

# Tokenize and Lemmatize list of words and join
lemmatized_tr = X_train.caption.map(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))
lemmatized_ts = X_test.caption.map(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))

In [0]:
tf_idf = TfidfVectorizer(max_features=3000, analyzer='word', stop_words= 'english', ngram_range=(1,3), dtype=np.float32)

tr = tf_idf.fit_transform(lemmatized_tr)
ts = tf_idf.transform(lemmatized_ts)

In [0]:
log_reg.fit(tr, y_train)
y_pred = log_reg.predict(ts)
accuracy_score(y_test, y_pred)

0.8105263157894737

# Captions and Labels tf-idf

In [0]:
reg_all_tfidf = post_df[['labels', 'caption', 'e_lev']]

reg_all_tfidf['labels_join'] = reg_all_tfidf.labels.map(lambda x: ' '.join(x))


X = reg_all_tfidf.drop(columns=['e_lev'])
Y = reg_all_tfidf['e_lev']

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.20, random_state=11)
log_reg = LogisticRegression(random_state = 20,class_weight = 'balanced')

In [0]:
lemmatizer = WordNetLemmatizer()

# Tokenize and Lemmatize list of words and join
lemmatized_tr = X_train.caption.map(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))
lemmatized_ts = X_test.caption.map(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))

In [0]:
tf_idf = TfidfVectorizer(max_features=3000, analyzer='word', stop_words= 'english', dtype=np.float32,  ngram_range=(1,3))

tr = tf_idf.fit_transform(lemmatized_tr)
ts = tf_idf.transform(lemmatized_ts)

In [0]:
tf_idf_l = TfidfVectorizer(max_features=300, analyzer='word', stop_words= 'english', dtype=np.float32, ngram_range=(1,3))

# lemmatized_tr_l = X_train.labels_join.map(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))
# lemmatized_ts_l = X_test.labels_join.map(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))

tr_l = tf_idf_l.fit_transform(X_train['labels_join']) #lemmatized_tr_l)#
ts_l = tf_idf_l.transform(X_test['labels_join']) #lemmatized_ts_l)

In [0]:
train = sparse.hstack([tr,tr_l])
test = sparse.hstack([ts,ts_l])

In [0]:
log_reg.fit(train, y_train)
y_pred = log_reg.predict(test)
accuracy_score(y_test, y_pred)

0.7789473684210526

# Topic Modeling

In [0]:
post_df.head(5)

,Unnamed: 0,post_url,num_likes,num_comments,caption,labels,engagement,e_lev
0,0,https://scontent-sea1-1.cdninstagram.com/vp/8e...,0.281916,0.094770,photo by jimmy chin @jimmychin | half dome aft...,"[mountainous landforms, mountain, white, black...",0.169628,1
1,1,https://scontent-sea1-1.cdninstagram.com/vp/51...,0.152775,0.072268,photos by ian teh @iantehphotography | i’m on ...,"[motor vehicle, vehicle, mode of transport, au...",0.104471,0
2,2,https://scontent-sea1-1.cdninstagram.com/vp/21...,0.377510,0.267337,photo by thomas peschak @thomaspeschak | the m...,"[green sea turtle, turtle, sea turtle, reptile...",0.311407,1
3,3,https://scontent-sea1-1.cdninstagram.com/vp/78...,0.099566,0.037785,photo by katie orlinsky @katieorlinsky | olga ...,"[temple, cuisine]",0.062497,0
4,4,https://scontent-sea1-1.cdninstagram.com/vp/03...,0.114799,0.056063,"photo by tasneem alsultan @tasneemalsultan | ""...","[tree, palm tree, arecales, woody plant, plant...",0.079557,0


In [0]:
import gensim
from gensim.utils import simple_preprocess
from gensim.test.utils import common_corpus, common_dictionary


In [0]:
# https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24
dictionary = gensim.corpora.Dictionary(post_df.labels)

dictionary.filter_extremes(no_below=1, no_above=0.5) #, keep_n=100000)

In [0]:
bow_corpus = [dictionary.doc2bow(label) for label in post_df.labels]

In [0]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=2, workers=2, chunksize=100, random_state=11)

In [0]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [0]:
pp.pprint(lda_model.print_topics())

[   (   0,
        '0.029*"tree" + 0.025*"plant" + 0.019*"grass" + 0.018*"sky" + '
        '0.015*"horse" + 0.015*"recreation" + 0.012*"photography" + '
        '0.012*"fun" + 0.012*"natural environment" + 0.010*"rock"'),
    (   1,
        '0.022*"adaptation" + 0.022*"sky" + 0.017*"wildlife" + '
        '0.015*"terrestrial animal" + 0.014*"mountain" + 0.013*"mammal" + '
        '0.013*"vertebrate" + 0.011*"human" + 0.011*"mountainous landforms" + '
        '0.010*"people"'),
    (   2,
        '0.028*"fish" + 0.027*"organism" + 0.025*"marine biology" + '
        '0.024*"underwater" + 0.020*"water" + 0.020*"vertebrate" + 0.016*"sky" '
        '+ 0.013*"wildlife" + 0.012*"blue" + 0.010*"cartilaginous fish"'),
    (   3,
        '0.044*"sky" + 0.031*"water" + 0.022*"photography" + 0.020*"landscape" '
        '+ 0.016*"sea" + 0.015*"wildlife" + 0.014*"natural landscape" + '
        '0.014*"nature" + 0.013*"cloud" + 0.012*"ocean"'),
    (   4,
        '0.030*"sky" + 0.023*"photography" + 0

In [0]:
post_topics = post_df.copy(deep=True)

post_topics['topic_0'] = 0.0
post_topics['topic_1'] = 0.0
post_topics['topic_2'] = 0.0
post_topics['topic_3'] = 0.0
post_topics['topic_4'] = 0.0

In [0]:
for index, row in post_topics.iterrows():
    bow = dictionary.doc2bow(row['labels'])
    t = lda_model.get_document_topics(bow)
    post_topics['topic_0'].loc[index] = t[0][1]
    post_topics['topic_1'].loc[index] = t[1][1]
    post_topics['topic_2'].loc[index] = t[2][1]
    post_topics['topic_3'].loc[index] = t[3][1]
    post_topics['topic_4'].loc[index] = t[4][1]


In [0]:
post_topics.head(3)

,Unnamed: 0,post_url,num_likes,num_comments,caption,labels,engagement,e_lev,topic_0,topic_1,topic_2,topic_3,topic_4
0,0,https://scontent-sea1-1.cdninstagram.com/vp/8e...,0.281916,0.094770,photo by jimmy chin @jimmychin | half dome aft...,"[mountainous landforms, mountain, white, black...",0.169628,1,0.018281,0.018656,0.018838,0.925465,0.018760
1,1,https://scontent-sea1-1.cdninstagram.com/vp/51...,0.152775,0.072268,photos by ian teh @iantehphotography | i’m on ...,"[motor vehicle, vehicle, mode of transport, au...",0.104471,0,0.020362,0.919250,0.020041,0.020154,0.020193
2,2,https://scontent-sea1-1.cdninstagram.com/vp/21...,0.377510,0.267337,photo by thomas peschak @thomaspeschak | the m...,"[green sea turtle, turtle, sea turtle, reptile...",0.311407,1,0.357763,0.018350,0.285217,0.320465,0.018204


In [0]:
labels = list(set([i for l in post_df.labels for i in l]))

word_loadings = pd.DataFrame(labels, columns=['word'])
word_loadings['topic_0'] = 0.0
word_loadings['topic_1'] = 0.0
word_loadings['topic_2'] = 0.0
word_loadings['topic_3'] = 0.0
word_loadings['topic_4'] = 0.0

In [0]:
word_loadings.head(1)

,word,topic_0,topic_1,topic_2,topic_3,topic_4
0,burmese python,0.0,0.0,0.0,0.0,0.0


In [0]:
for index, row in word_loadings.iterrows():
  try:
    w = lda_model.get_term_topics(row['word'], minimum_probability=0.000001)
    word_loadings['topic_0'].loc[index]  = w[0][1]
    word_loadings['topic_1'].loc[index]  = w[1][1]
    word_loadings['topic_2'].loc[index]  = w[2][1]
    word_loadings['topic_3'].loc[index]  = w[3][1]
    word_loadings['topic_4'].loc[index]  = w[4][1]
  except:
    pass

In [0]:
word_loadings[word_loadings['topic_0']==0.0].count()

word       0
topic_0    0
topic_1    0
topic_2    0
topic_3    0
topic_4    0
dtype: int64

In [0]:
word_loadings.head(5)

,word,topic_0,topic_1,topic_2,topic_3,topic_4
0,burmese python,0.000128,0.000005,0.000005,0.001291,0.000007
1,room,0.001197,0.003754,0.000604,0.008210,0.005228
2,valley,0.000006,0.000639,0.000005,0.000007,0.000007
3,webbing clothes moth,0.000006,0.000005,0.000005,0.000341,0.000007
4,pomacentridae,0.000006,0.000005,0.000547,0.000003,0.000007


# Quartiles

In [0]:
post_topics['quartile'] = pd.qcut(post_topics['engagement'], 4, labels=False)

In [0]:
post_topics.head(2)

,Unnamed: 0,post_url,num_likes,num_comments,caption,labels,engagement,e_lev,topic_0,topic_1,topic_2,topic_3,topic_4,quartile
0,0,https://scontent-sea1-1.cdninstagram.com/vp/8e...,0.281916,0.094770,photo by jimmy chin @jimmychin | half dome aft...,"[mountainous landforms, mountain, white, black...",0.169628,1,0.018281,0.018656,0.018838,0.925465,0.018760,2
1,1,https://scontent-sea1-1.cdninstagram.com/vp/51...,0.152775,0.072268,photos by ian teh @iantehphotography | i’m on ...,"[motor vehicle, vehicle, mode of transport, au...",0.104471,0,0.020362,0.919250,0.020041,0.020154,0.020193,1


In [0]:
post_topics.groupby('quartile')[['topic_0','topic_1','topic_2','topic_3','topic_4']].mean()

,topic_0,topic_1,topic_2,topic_3,topic_4
quartile,,,,,
0,0.227681,0.163748,0.138967,0.262377,0.207228
1,0.127168,0.256484,0.152151,0.280183,0.184014
2,0.183308,0.208172,0.142312,0.364503,0.101704
3,0.114926,0.249348,0.197415,0.344589,0.093721


# Visualization

In [0]:
pip install pyLDAvis

In [0]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.013521  0.049232       1        1  32.735279
1      0.040383  0.011363       2        1  20.839817
0      0.047630 -0.145210       3        1  16.201387
2     -0.160416 -0.003691       4        1  15.904014
4      0.058882  0.088306       5        1  14.319504, topic_info=    Category       Freq            Term      Total  loglift  logprob
251  Default  19.000000            fish  19.000000  30.0000  30.0000
21   Default  18.000000  marine biology  18.000000  29.0000  29.0000
28   Default  21.000000      underwater  21.000000  28.0000  28.0000
24   Default  30.000000        organism  30.000000  27.0000  27.0000
37   Default  30.000000           plant  30.000000  26.0000  26.0000
..       ...        ...             ...        ...      ...      ...
103   Topic5   4.856909           ocean  30.245243   0.1146  -4.8222
38    Topic5   5.375129            tree  49.329121  -0.2732  -4.7208
66    Topic5   4.244387          nature  30.866831  -0.0405  -4.9570
262   Topic5   3.820674           green  15.850994   0.5207  -5.0622
41    Topic5   3.551536           human  22.196745   0.1110  -5.1352

[343 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
19        1  0.286766   adaptation
19        2  0.441178   adaptation
19        3  0.132353   adaptation
19        4  0.132353   adaptation
19        5  0.022059   adaptation
...     ...       ...          ...
39        1  0.112117  woody plant
39        2  0.448466  woody plant
39        3  0.448466  woody plant
491       2  0.796494        world
491       4  0.199123        world

[544 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 1, 3, 5])